## Acoustic Integration - Insulation Size Estimation

http://localhost:8888/?token=sloth

In [ ]:
if "mi" not in vars():
    visual = False

    import numpy as np
    from tqdm import trange
    import matplotlib.pyplot as plt

    if not visual:
        %matplotlib ipympl

    import drjit as dr
    import mitsuba as mi

    from libs import utils, acoustic_torch

    plt.style.use('ggplot')
    mi.set_log_level(mi.LogLevel.Warn)
    utils.drjit_turn_off_optimizations(False)

    if visual:
        mi.set_variant('cuda_ad_rgb')
    else:
        mi.set_variant('cuda_ad_acoustic')

    sess_seed   = np.random.randint(0, 2**30)
    sess_seed_g = np.random.randint(0, 2**30)
    print(f"session seeds are: sess_seed={sess_seed}; sess_seed_g={sess_seed_g}")

### Scene Construction

In [ ]:
def shoebox_scene_wrapper(visual=False, **kwargs):
    tf          = mi.ScalarTransform4f
    box_dim     = np.array(kwargs['box_dim']) / 2.
    eps         = 100. * dr.epsilon(mi.Float)

    scene = utils.shoebox_scene_visual(**kwargs) if visual else utils.shoebox_scene(**kwargs)
    scene["eierpappe"] = {
        "type": "ply",
        "filename": "/home/daniel/Studium/masterarbeit/data/scenes/meshes/rectangle.ply",
        "bsdf": {
            "type": "acousticbsdf",
            "scattering": { "type": "spectrum", "value": kwargs['scattering'] },
            "absorption": { "type": "spectrum", "value": 0.9 },
        },
        "to_world": tf.translate([0., 0., eps - box_dim[2]]).scale([6., box_dim[1] - 0.5, 1.])
    }

    if visual:
        scene["eierpappe"]["bsdf"] = { "type": "diffuse", "reflectance": 0.9 }
        scene["integrator"] = {
            "type": "prb_reparam",
            "max_depth": 8,
            "hide_emitters": True,
            # "reparam_rays": 32,
        }
    else:
        scene["integrator"]["skip_direct"] = True

    return scene

In [ ]:
absorption = [0.1, 0.2]
# scattering = [0.2, 0.8]

# absorption_mdim, scattering_mdim = np.meshgrid(absorption, scattering)
# absorption,      scattering      = absorption_mdim.flatten(), scattering_mdim.flatten()

config = {
    "box_dim":     [25., 12., 7.],
    "mic_pos":     [ 9.,  6., 1.],
    "speaker_pos": [20.,  7., 2.],
    "speaker_radius": 0.5, #0.1,

    "absorption": [(i + 1, a) for i, a in enumerate(absorption)],
    "scattering": 0.8, #[(i + 1, s) for i, s in enumerate(scattering)],

    "wav_bins":  len(absorption), # x
    "time_bins": 60,             # y
    "max_time":  1.5,

    # "integrator": "prb_acoustic",
    "integrator": "prb_reparam_acoustic",
    "max_depth": 60,
    "spp": 2**18,
}

fs = config["time_bins"] / config["max_time"]
time = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)

max_depth_rec       = utils.estimate_max_depth(config["box_dim"], config["max_time"], 1.5)
# config["max_depth"] = max_depth_rec
print(f"max_depth = {config['max_depth']}; should be = {max_depth_rec}")

### Reference Histogram

In [ ]:
scene_dict = shoebox_scene_wrapper(visual=visual, **config)
scene      = mi.load_dict(scene_dict)
img_ref    = mi.render(scene, seed=sess_seed)

if visual:
    utils.plot_img(img_ref)
else:
    utils.plot_hist(img_ref[:, :, 0], **config)

### Optimization Setup

In [ ]:
params = mi.traverse(scene)
# display(params)

key = 'eierpappe.vertex_positions'
vertex_pos_ref = dr.unravel(mi.Point3f, params[key])
display(vertex_pos_ref)

In [ ]:
opt = mi.ad.Adam(lr=0.025)
opt['s'] = mi.Point3f(1.5, 1.0, 1.0)

def apply_transform():
    opt['s']   = dr.clamp(opt['s'], 0.0, 2.0)
    opt['s'].y = opt['s'].z = 1.0

    params[key] = dr.ravel(vertex_pos_ref * (2. - opt['s']))
    # params[key] = dr.ravel(vertex_pos_ref * opt['s'])
    params.update()

In [ ]:
apply_transform()
img = mi.render(scene, seed=sess_seed)
if visual:
    utils.plot_img(img)
else:
    utils.plot_hist(img[:, :, 0], **config)

### Visualize Loss/Gradient-Spectogram

In [ ]:
edc_ref = acoustic_torch.EDC(img_ref[:, :, 0], db=True, norm=True)
t30_ref = acoustic_torch.T(mi.TensorXf(time), edc_ref)

def loss(a, b):
    # a = acoustic_torch.C(a[:, :, 0], fs=fs)
    # b = acoustic_torch.C(b[:, :, 0], fs=fs)
    # a = acoustic_torch.D(a[:, :, 0], fs=fs)
    # b = acoustic_torch.D(b[:, :, 0], fs=fs)
    # a = acoustic_torch.TS(mi.TensorXf(time), a[:, :, 0])
    # b = acoustic_torch.TS(mi.TensorXf(time), b[:, :, 0])

    a = acoustic_torch.EDC(a[:, :, 0], db=True, norm=True)
    # b = acoustic_torch.EDC(b[:, :, 0], db=True, norm=True)
    a = acoustic_torch.T(mi.TensorXf(time), a)
    # b = acoustic_torch.T(mi.TensorXf(time), b)
    b = t30_ref

    return utils.mse(a, b)

In [ ]:
opt['s'] = mi.Point3f(1.1, 1., 1.)
apply_transform()
img = mi.render(scene, seed=sess_seed)
dr.enable_grad(img)
l = loss(img, img_ref)
dr.backward(l)

gradl = mi.TensorXf(dr.grad(img))
print(dr.min(gradl), dr.max(gradl))

if visual:
    gradl = dr.maximum(0., gradl - dr.min(gradl))
    gradl = dr.minimum(1., gradl / dr.max(gradl))
    utils.plot_img(gradl)
else:
    utils.plot_hist(gradl[:, :, 0], **config)

In [ ]:
if False and not visual:
    factors = np.linspace(0.5, 1.5, 11, endpoint=True)
    losses  = np.zeros((factors.shape[0], 2))
    grads   = np.zeros((factors.shape[0], 2))

    assert losses.shape == grads.shape

    for i in trange(losses.shape[0]):
        opt['s'] = mi.Point3f(factors[i], 1., 1.)
        apply_transform()
        img = mi.render(scene, params, seed=i+1, seed_grad=i+2)

        l0  = dr.sqr(dr.sum(img[:, :, 0]) - dr.sum(img_ref[:, :, 0]))
        dr.backward(l0)
        g0  = dr.grad(opt['s']).x

        losses[i, 0] = l0[0]
        grads[i, 0]  = g0[0]


        opt['s'] = mi.Point3f(factors[i], 1., 1.)
        apply_transform()
        img = mi.render(scene, params, seed=i+1, seed_grad=i+2)

        # grad = mi.TensorXf(np.array([
        #     [[0., 0.], [0., 0.]],
        #     [[0., 0.], [0., 0.]],
        #     [[2., 0.], [2., 0.]],
        #     [[2., 0.], [2., 0.]],
        #     [[0., 0.], [0., 0.]],
        # ])) * dr.sign(mi.Float(factors[i])) * -1000.
        # dr.set_grad(img, grad)
        # g1 = dr.backward_to(opt['s'])

        l1  = mse(img, img_ref)
        dr.backward(l1)
        g1  = dr.grad(opt['s']).x

        losses[i, 1] = l1[0]
        grads[i, 1]  = g1[0]

    fig, axis = plt.subplots(2, 2, figsize=(12, 8), sharex=True, sharey=False)
    axf = axis.flatten()

    axf[0].plot(factors, losses[:, 0])
    axf[1].plot(factors, losses[:, 1])
    axf[2].plot(factors, grads[:, 0])
    axf[3].plot(factors, grads[:, 1])

    axf[2].axhline(0., -10., 10, c="C1", linestyle="dotted")
    axf[2].axvline(1., -2**30, 2**30, c="C1", linestyle="dotted")
    axf[3].axhline(0., -10., 10, c="C1", linestyle="dotted")
    axf[3].axvline(1., -2**30, 2**30, c="C1", linestyle="dotted")

    axf[0].set_title("$\\left(\sum_{i=1}^n x_i - \sum_{i=1}^n \overline{x}_i \\right)^2$")
    axf[1].set_title("$\\frac{1}{n}\sum_{i=1}^n (x_i - \overline{x}_i)^2$")

    axf[0].set_ylabel("Loss")
    axf[2].set_ylabel("Gradient opt['s']")

    # plt.close()
    fig.show()

### Main Loop

In [ ]:
opt['s'] = mi.Point3f(1.5, 1., 1.)

vals, losses, grads = [], [], []

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(12, 3), sharex=True)
ax1.set_title("values")
ax1.set_xlim(-1, 51)
ax1.set_ylim(-0.1, 2.1)

ax2.set_title("grads")
ax2.set_ylim(-1.1, 1.1)

ax3.set_title("losses");

In [ ]:
iters = 50
if iters > 1:
    n  = len(vals) + iters

# for i in trange(iters) if iters > 1 else range(iters):
for i in range(iters) if iters > 1 else range(iters):
    apply_transform()
    if visual:
        img = mi.render(scene, params,
                        seed=sess_seed+i, seed_grad=sess_seed_g+i, spp=25)
    else:
        img = mi.render(scene, params,
                        seed=sess_seed+i, seed_grad=sess_seed_g+i)

    l = loss(img, img_ref)
    dr.backward(l, flags=dr.ADFlag.ClearNone if iters < 2 else dr.ADFlag.Default)
    # dr.set_grad(opt['s'], mi.Point3f(100., 0., 0.))

    if iters < 2:
        display(dr.unravel(mi.Point3f, dr.grad(params[key])))
        display(dr.grad(opt['s']).x)
    else:
        losses.append(l[0])
        vals.append(opt['s'].x[0])
        grads.append(dr.grad(opt['s']).x[0])

        opt.step()

        ax1.clear()
        ax1.set_title("values")
        ax1.plot(vals)
        ax1.hlines([1.0], 0, n, colors='k', linestyles='dotted')
        ax1.set_xlim(-n * 0.02, n * 1.02)
        ax1.set_ylim(-0.1, 2.1)

        ax2.clear()
        ax2.set_title("grads")
        ax2.plot(grads)

        ax3.clear()
        ax3.set_title("losses")
        ax3.plot(losses)

        fig.canvas.draw()

In [ ]:
if iters > 1:
    losses = np.array(losses)
    vals   = np.array(vals)
    grads  = np.array(grads)
    data = np.stack([losses, vals, grads])
    # np.save("/home/daniel/reparam-eierpappe-plain-prb-t30-1_5.npy", data)

### Estimated (Forward) Gradients vs. Finite Differences

In [ ]:
randint = np.random.randint(0, 2**20)

# forward_pass
opt['s'] = mi.Point3f(0.2, 0.0, 0.0)
apply_transform()

assert dr.grad_enabled(opt['s'])
dr.set_grad(opt['s'], mi.Point3f(1.0, 0.0, 0.0))

img  = mi.render(scene, params, seed=randint, seed_grad=randint+1)
grad = dr.forward_to(img)

# finite differences
opt['s'] = mi.Point3f(0.2, 0.0, 0.0)
apply_transform()
img = mi.render(scene, seed=randint)

opt['s'] = mi.Point3f(0.2 + 1e-2, 0.0, 0.0)
apply_transform()
finite_difference = mi.render(scene, seed=randint) - img
# finite_difference = img - mi.render(scene, seed=randint)

In [ ]:
if not visual:
    t = np.linspace(0., config["max_time"], config["time_bins"], endpoint=False)
    label = list(map(lambda x: f"$\\alpha={x:.2f}$", absorption))

    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 4), sharey=False)

    ax1.plot(t, grad[:, :, 0].numpy())
    ax1.set_xlabel("time [s]")
    ax1.set_ylabel("Energy [a.U.]")
    ax1.set_title("PRB Reparam Gradient")

    ax2.plot(t, finite_difference[:, :, 0].numpy(), label=label)
    ax2.set_xlabel("time [s]")
    ax2.set_title("finite differences")
    ax2.legend()

    plt.show()
else:
    _, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

    ax1.imshow(bitmap(grad, convert=True))
    ax1.axis("off")
    ax1.set_title("PRB Reparam Gradient")

    ax2.imshow(bitmap(dr.abs(finite_difference), convert=True))
    ax2.axis("off")
    ax2.set_title("finite differences")

    plt.show()

    grad_t              = grad.numpy().transpose(1, 0, 2)
    finite_difference_t = finite_difference.numpy().transpose(1, 0, 2)

    _, axis = plt.subplots(3, 2, figsize=(12, 12), sharex=True)

    axis[0, 0].set_title("PRB Reparam Gradient")
    axis[0, 0].plot(grad_t[:, :, 0].flatten())
    axis[1, 0].plot(grad_t[:, :, 0].flatten())
    axis[2, 0].plot(grad_t[:, :, 0].flatten())

    axis[0, 1].set_title("finite differences")
    axis[0, 1].plot(finite_difference_t[:, :, 0].flatten())
    axis[1, 1].plot(finite_difference_t[:, :, 0].flatten())
    axis[2, 1].plot(finite_difference_t[:, :, 0].flatten())

    plt.show()